# Ball Following Bot
## This notebook is a part of Ball Following Bot project.This notebook shows how a Raspberry Pi is interfaced with PC for live-streaming of video from Pi to PC for further processing of the video by OpenCV to detect the ball.
## The Pi also sends the distance of the ball (feeded from ultrasonic sensor which itself is mounted on a rotating Servo motor) and the angle of rotation (of the servo motor) to the PC. The distance of the ball and the angle of rotation of motor can be used explicitly to determine  the location of the ball in a Radar-like detection system in PC.
![radar](https://imgur.com/ldJxmfc.jpeg)
## For transportation of information b/w PC and Pi I have created a simple PHP server on my PC to send data via GET method and then to store it in a txt file to be retrieved by the required program. 
*(The PHP code is given at the end of this notebook)*

## The basic prototype model of the Bot:-
![bot1](https://imgur.com/5U5wDeH.jpeg)
![bot2](https://imgur.com/77gQOBp.jpeg)
![bot3](https://imgur.com/8JZV0Fy.jpeg)

# The code has two parts , one has to be run on PC and another on Pi.

## PC code:- 
##### N.B.- The code has to be run in seperate .py files from the terminal simulataneously and not from the notebook directly.

** This code is to implement the Radar System on PC. **

In [ ]:
#pcradar.py
import pygame
import math
import sys
from pygame.locals import *

pygame.init()
width,height=700,700
disp=pygame.display.set_mode((width,height))
#ser=serial.Serial()
#ser.timeout=1
#ser.port='com6' # port name where the arduino 
#ser.open()
green=(0,255,0)
point_list=[]

##convert functions
def to_window(x,y): # coordinate to window
    x,y=int(x),int(y)
    n_x=x+width//2
    n_y=height//2 -y
    return([n_x,n_y])

def to_radian(angle):
    x=(angle*3.14)/180
    return(x)
##

##draw function
def add_to_list(angle,distance):
    global point_list
    distance*=10
    angle=to_radian(angle)
    x,y=math.cos(angle)*distance,math.sin(angle)*distance
    pos=to_window(x,y)
    point_list.append(pos)
    
def draw_point(list_point,disp=disp):
    for point in list_point:
        pygame.draw.circle(disp,green,point,2)

def draw_circles(disp=disp):
    raduis=50
    for x in range(1,width//2):
        n_raduis=((x//raduis)+1)*raduis
        pygame.draw.circle(disp,green,(width//2,height//2),n_raduis,2)
def draw_line(angle,disp=disp):
    a=math.tan(to_radian(angle))
    if angle<=180:
        y=height//2
    elif angle>180:
        y=-height//2
    if a==0:
        y=0
        if angle==0:x=width/2
        elif angle==180:x=-width/2
    else:x=y//a
    pos=to_window(x,y)
    pygame.draw.line(disp,green,(width/2,height/2),pos,2)

def angle_distance(ser):
    #ser.reset_input_buffer()
    read_input=ser.readline()
    read_input=read_input.decode()
    read_input=read_input[:len(read_input)-2]
    read_input=read_input.split(";")
    if len(read_input)!=2 or ("" in read_input):
        return(False)
    return(read_input)

def draw_text(disp,text,t_size):
        fontObj = pygame.font.Font('freesansbold.ttf',t_size)
        textSurface=fontObj.render(text, True,(255,255,255))
        disp.blit(textSurface,(0,0))
        
##main##
while True:
    disp.fill((0,0,0))
    file=open('C:\\xampp\\htdocs\\try\\ussensor.txt','r')
    s=file.read()
    i=0
    ang=[]
    dist=[]
    angle=''
    distance=''
    try:
        while s[i]!=' ':
            ang.append(s[i])
            i=i+1
    except:
        continue
    i=i+1
    while i<len(s):

        dist.append(s[i])
        i=i+1
    for j in range(0,len(ang)):
        angle=angle+str(ang[j])
    for j in range(0,len(dist)):
        distance=distance+str(dist[j])
    #print(angle)
    #print(distance)
    file.close()
    angle=float(angle)
    distance=float(distance)
    #angle_dist=angle_distance(ser)
    #if angle_dist==False:continue
    if (angle==0):point_list=[]
    draw_circles(disp)
    draw_line(angle)
    add_to_list(angle,distance)
    draw_point(point_list)
    draw_text(disp,"angle = "+str(angle)+", distance = "+str(distance),15)
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    pygame.display.update()

** Code for processing the live-streamed video from the Pi using OpenCV .The code also finds the X-axis coordinate of the the ball position in the current frame and saves it in a text file to be retrieved by the Pi.  **
__Install the necessary packages before runing the program.__

In [ ]:
#ball_tracking.py

# import the necessary packages
from collections import deque
import numpy as np
import argparse
import imutils
import cv2

# define the lower and upper boundaries of the "green"
# ball in the HSV color space
greenLower = (19, 51, 17)
greenUpper = (49, 253, 227)
yellowLower=(16,143,36)
yellowUpper=(29,255,197)

# initialize the list of tracked points, the frame counter,
# and the coordinate deltas
pts = deque(maxlen=32)
counter = 0
(dX, dY) = (0, 0)
direction = ""

#Change the IP address below to the IP address of your Pi connected to same local network with the PC
camera = cv2.VideoCapture("tcp://192.168.43.60:8001/")

# keep looping
while (cv2.waitKey(1)!=27 and camera.isOpened()) :
    # grab the current frame
    (grabbed, frame) = camera.read()

	# resize the frame, blur it, and convert it to the HSV
	# color space
    frame = imutils.resize(frame, width=640)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

	# construct a mask for the color "green", then perform
	# a series of dilations and erosions to remove any small
	# blobs left in the mask
    mask = cv2.inRange(hsv, yellowLower, yellowUpper)
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)

	# find contours in the mask and initialize the current
	# (x, y) center of the ball
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]
    center = None

	# only proceed if at least one contour was found
    if len(cnts) > 0:
		# find the largest contour in the mask, then use
		# it to compute the minimum enclosing circle and
		# centroid
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

		# only proceed if the radius meets a minimum size
        if radius > 1:
			# draw the circle and centroid on the frame,
			# then update the list of tracked points
            cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
            cv2.circle(frame, center, 5, (0, 0, 255), -1)
            pts.appendleft(center)

	# loop over the set of tracked points


	# show the movement deltas and the direction of movement on
	# the frame
    cv2.putText(frame, direction, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,0.65, (0, 0, 255), 3)
    cv2.putText(frame, "dx: {}, dy: {}".format(int(x), int(y)),(10, frame.shape[0] - 10), cv2.FONT_HERSHEY_SIMPLEX,0.35, (0, 0, 255), 1)
    

	# show the frame to our screen and increment the frame counter
    cv2.imshow("Frame", frame)
    try:
        file=open('C:\\xampp\\htdocs\\try\\xcoor.txt','w')
    except OSError:
        pass
    try:
        file.write(str(x))
    except ValueError:
        pass
    file.close()
    key = cv2.waitKey(1) & 0xFF
    counter += 1

	# if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break

# cleanup the camera and close any open windows
camera.release()
cv2.destroyAllWindows()

## Pi code:-
##### N.B.- The code has to be run in seperate .py files inside the Raspberry Pi.

** This code actually has to send the the angle of rotation of the Servo motor and Distance of the the ball as measured by the Ultrasonic Sensor. But for testing purpose I have passed a random integer as distance and continuously increasing angle.**

In [ ]:
#pi code1 radardata.py
import requests
import random
import time
ip=input('Enter IP address of the PC:')
while True:
    for i in range(0,360):
        time.sleep(0.15)
        angle=i
        distance=random.randint(0,100)
        url='http://'+ip+'/try/sensor.php?angle='+str(angle)+'&distance='+str(distance)
        file=open('angle.txt','w')
        response=requests.get(url)
        file.write(str(angle))
        file.close()
        #print(response.status_code)

** This code is responsible for creating the Camera Server in the Pi.The video frames produced here are then retrieved by ball_tracking.py program in PC.**

In [ ]:
#pi code2 cameraserver.py
import socket
import time
import picamera

camera = picamera.PiCamera()
camera.resolution = (640, 480)
#camera.resolution = (1280, 720)
camera.framerate = 24

server_socket = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
server_socket.bind(('0.0.0.0', 8001))

server_socket.setsockopt( socket.SOL_SOCKET,socket.SO_REUSEADDR, 1 )
server_socket.listen(0)
# Accept a single connection and make a file-like object out of it
connection = server_socket.accept()[0].makefile('wb')
try:
    camera.start_recording(connection, format='mjpeg')
    camera.wait_recording(10000)
    camera.stop_recording()
finally:
    connection.close()
    server_socket.close()


** This code reads the X-Coordinate of the ball on the frame as stored by the 'ball_tracking.py' program in PC for further determining the course of action by the Pi (incomplete).**

In [ ]:
#pi code3 readinstructions.py
import requests
import time
ip='192.168.43.208'
url='http://'+ip+'/try/xcoor.txt'
while True:
    time.sleep(0.1)
    file=open('angle.txt','r')
    angle=file.read()
    file.close()
    try:
        response=requests.get(url)
        x=response.text
        print('xcoor='+x)
        print('angle='+angle)
    except requests.exceptions.ConnectionError:
        print('...')
        pass

** PHP Code to be stored in HTDocs folder.**

In [ ]:
# PHP Code
#sensor.php
<?php
$angle=$_GET['angle'];
$distance=$_GET['distance'];
$file=fopen('ussensor.txt','w');
$text=$angle." ".$distance;
fwrite($file, $text);
fclose($file);
?>

## So here is the end result:-
![finalbot](https://imgur.com/XPLBKO3.jpeg)
### and the working ball detection video is [here](https://drive.google.com/file/d/1grI2u22Evk3Peo35_V-BolNTVE3BMrsI/view?usp=sharing)
### and radar system video is [here](https://drive.google.com/file/d/1uv8n0i-IP7oE5npESHBV_pnjMz_XuL1L/view?usp=sharing).